In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2013.json"

response = requests.get(URL)
with open("index_2013.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2013.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.head(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
0,205585992,201306,93492296006093,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,PTOK BUSH ELEMENTARY INC,2013-12-04,201342969349200609,2016-03-21T17:23:53
1,236437027,201306,93492296007043,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,SOCIETY OF THE ARTS CO ALLENTOWN ART MUSEUM,2013-12-04,201342969349200704,2016-03-21T17:23:53
2,452096099,201212,93492296007093,990EZ,https://s3.amazonaws.com/irs-form-990/20134296...,GREAT FOUNDATIONS INC,2013-12-04,201342969349200709,2016-03-21T17:23:53
3,731664822,201212,93492310012033,990EZ,https://s3.amazonaws.com/irs-form-990/20133310...,FRIENDS OF KIDS INC,2013-12-04,201333109349201203,2016-03-21T17:23:53
4,200128337,201306,93492310003043,990EZ,https://s3.amazonaws.com/irs-form-990/20134310...,LAKESHORE YOUTH ATHLETIC ASSOC,2013-12-04,201343109349200304,2016-03-21T17:23:53


In [5]:
df.shape

(261449, 9)

In [6]:
from multiprocessing.pool import ThreadPool

In [7]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2013/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [8]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2013/'):
    f.extend(filenames)
    break

In [9]:
u = list(df['URL'])

In [10]:
len(u)

261449

In [11]:
file_names_list = []

In [12]:
len(file_names_list)

0

In [13]:
for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [14]:
len(file_names_list)

261449

In [15]:
len(f)

261274

In [16]:
diff = list(set(file_names_list) - set(f))

In [17]:
len(diff)

160

In [18]:
diff[0]

'201301149349300210_public.xml'

In [19]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [20]:
url_pre

'https://s3.amazonaws.com/irs-form-990/'

In [21]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [22]:
urls[0]

'https://s3.amazonaws.com/irs-form-990/201301149349300210_public.xml'

In [23]:
for url in urls:
    download_url(url)

In [24]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2013/'):
    f.extend(filenames)
    break

In [25]:
len(f)

261434

In [26]:
len(file_names_list)


261449

In [27]:
diff = list(set(file_names_list) - set(f))

In [28]:
len(diff)

0

In [29]:
diff = set(file_names_list).symmetric_difference(set(f))

In [30]:
len(diff)

0

In [31]:
diff

set()

In [32]:
dff = list(set(f) - set(file_names_list))

In [33]:
len(dff)

0

In [34]:
len(file_names_list)

261449

In [35]:
len(f)

261434

In [36]:
len(list(set(file_names_list)))

261434

In [37]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [38]:
print(list_duplicates(file_names_list))

[71821, 71822, 71823, 71824, 71826, 71827, 71828, 71829, 71830, 71831, 183840, 217142, 217143, 217144, 257543]


In [39]:
df.iloc[71821,]

EIN                                                         141910988
TaxPeriod                                                      201212
DLN                                                    93492122002013
FormType                                                        990EZ
URL                 https://s3.amazonaws.com/irs-form-990/20131122...
OrganizationName                              HIV DENTAL ALLIANCE INC
SubmittedOn                                                2013-07-25
ObjectId                                           201311229349200201
LastUpdated                                       2016-03-21T17:23:53
Name: 71821, dtype: object

In [41]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [48]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [49]:
dup_ein

,EIN
71821,141910988
71822,311451834
71823,237448364
71824,222802132
71826,341397769
71827,263115088
71828,202787636
71829,133944085
71830,251520627
71831,341280845


In [52]:
dup_ein.iloc[0]['EIN']

'141910988'

In [53]:
dup_ein.shape

(15, 1)

In [55]:
ein = dup_ein.iloc[0]['EIN']
test = df[df['EIN'] == ein]

In [56]:
test

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
69719,141910988,201212,93492122002013,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,HIV DENTAL ALLIANCE INC,2013-07-24,201311229349200201,2016-03-21T17:23:53
71821,141910988,201212,93492122002013,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,HIV DENTAL ALLIANCE INC,2013-07-25,201311229349200201,2016-03-21T17:23:53


In [57]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [59]:
duplicates.shape

(30, 9)

In [60]:
duplicates

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
69719,141910988,201212,93492122002013,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,HIV DENTAL ALLIANCE INC,2013-07-24,201311229349200201,2016-03-21T17:23:53
71821,141910988,201212,93492122002013,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,HIV DENTAL ALLIANCE INC,2013-07-25,201311229349200201,2016-03-21T17:23:53
69720,311451834,201212,93492122002063,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,I AM SOMEBODY,2013-07-24,201311229349200206,2016-03-21T17:23:53
71822,311451834,201212,93492122002063,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,I AM SOMEBODY,2013-07-25,201311229349200206,2016-03-21T17:23:53
69721,237448364,201212,93492122002113,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,PIER ONE THEATRE INC,2013-07-24,201311229349200211,2016-03-21T17:23:53
71823,237448364,201212,93492122002113,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,PIER ONE THEATRE INC,2013-07-25,201311229349200211,2016-03-21T17:23:53
69722,222802132,201303,93492122002163,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,MARPLE-NEWTON SOCCER ASSOCIATION INC,2013-07-24,201311229349200216,2016-03-21T17:23:53
71824,222802132,201303,93492122002163,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,MARPLE-NEWTON SOCCER ASSOCIATION INC,2013-07-25,201311229349200216,2016-03-21T17:23:53
69723,341397769,201212,93492122002313,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,BELLEVUE DEVELOPMENT CORP,2013-07-24,201311229349200231,2016-03-21T17:23:53
71826,341397769,201212,93492122002313,990EZ,https://s3.amazonaws.com/irs-form-990/20131122...,BELLEVUE DEVELOPMENT CORP,2013-07-25,201311229349200231,2016-03-21T17:23:53
